In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import seaborn as sns
import numpy as np
import math
from sklearn import datasets, linear_model, model_selection
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix
import datetime as dt
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler

path = (r'~/Study in MISIS/MISIS-master/notebooks/season_2/dz_2/athletes_sochi.txt')
dfs = pd.read_csv(path).dropna()
alldata = dfs[['birthdate', 'weight', 'height', 'gender']]

def get_data(features, predicted):
    now = pd.Timestamp('now')
    new_gender = {"gender": {"Male" : 1, "Female" : 0}}
    newdata = dfs.replace(new_gender)
    now = pd.Timestamp('now')
    newdata['birthdate'] = pd.to_datetime(dfs['birthdate'], format='%Y-%m-%d')
    newdata['birthdate'] = newdata['birthdate'].where(newdata['birthdate'] < now, newdata['birthdate'] -  np.timedelta64(100, 'Y'))
    newdata['age'] = (now - newdata['birthdate']).astype('<m8[Y]')
    newdata["BMI"] = newdata["weight"] / (newdata["height"]**2)
    newdata = newdata.sample(frac=1)
    global allinput, alloutput
    allinput = newdata[features]
    alloutput = newdata[predicted]
    print(allinput, alloutput)

def train_and_test_model_linear():
    global X_train, X_test, Y_train, Y_test
    X_train, X_test, Y_train, Y_test = model_selection.train_test_split(allinput, alloutput, test_size=0.05)

    global model
    model = linear_model.LogisticRegression()
    model.fit(X_train, Y_train)
    global Y_test_predicted
    Y_test_predicted = model.predict(X_test)

def get_sex(myW, myH, myA):
    myBMI=myW/(myH**2)
    mysex=model.predict_proba([[myW, myH, myA, myBMI]])
    print('\n', mysex)

def precision_and_recall():
    [tn, fp], [fn, tp] = confusion_matrix(Y_test, Y_test_predicted)
    confusion_matrix(Y_test, Y_test_predicted)
    precision = tp / (tp + fp)
    recall = tp / tp + fn

    print('\n', "Precision: ", tp / (tp + fp))
    print("Recall: ", tp / (tp + fn))
    print("Accuracy: ", (tn + tp) / (tp + fp + tn + fn))
    print("F1: ",  (precision * recall) / ((precision) + recall))
    report = classification_report(Y_test, model.predict(X_test), target_names=['Male', 'Female'])
    print(report)

get_data(features = ['weight', 'height', 'age', 'BMI'], predicted = 'gender')
train_and_test_model_linear()

precision_and_recall()
#Accuracy увеличивается за счет добавления BMI и возраста в модель, однако тогда в модели появляется мультиколлинеарность.
#Также accuracy увеличивается в зависимости от test_size датасета

      weight  height   age        BMI
2319    62.0    1.73  31.0  20.715694
1179    58.0    1.71  38.0  19.835163
2252    86.0    1.81  36.0  26.250725
493     43.0    1.52  24.0  18.611496
2742    91.0    1.88  29.0  25.746944
...      ...     ...   ...        ...
2427    93.0    1.85  40.0  27.173119
2054    78.0    1.82  32.0  23.547881
1365    95.0    1.83  33.0  28.367524
1605    60.0    1.65  28.0  22.038567
2180    70.0    1.77  29.0  22.343516

[2479 rows x 4 columns] 2319    0
1179    1
2252    1
493     0
2742    1
       ..
2427    1
2054    1
1365    1
1605    1
2180    1
Name: gender, Length: 2479, dtype: int64

 Precision:  0.8513513513513513
Recall:  0.8873239436619719
Accuracy:  0.8467741935483871
F1:  0.7777777777777778
              precision    recall  f1-score   support

        Male       0.84      0.79      0.82        53
      Female       0.85      0.89      0.87        71

    accuracy                           0.85       124
   macro avg       0.85      0.84  